In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import torchvision

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Generator and Discriminator networks
class Generator(nn.Module):
    def __init__(self, input_size, output_size):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, output_size),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Hyperparameters
input_size = 100
batch_size = 64
num_epochs = 30
learning_rate = 0.0002

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize networks and optimizers
generator = Generator(input_size, 784).to(device)
discriminator = Discriminator(784).to(device)
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate)

# Loss function
criterion = nn.BCELoss()

# Training loop
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        real_images = real_images.view(-1, 784).to(device)
        batch_size = real_images.size(0)

        # Training Discriminator
        optimizer_D.zero_grad()
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Real images
        real_outputs = discriminator(real_images)
        real_loss = criterion(real_outputs, real_labels)

        # Fake images
        z = torch.randn(batch_size, input_size).to(device)
        fake_images = generator(z)
        fake_outputs = discriminator(fake_images.detach())
        fake_loss = criterion(fake_outputs, fake_labels)

        # Total loss
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Training Generator
        optimizer_G.zero_grad()
        fake_outputs = discriminator(fake_images)
        g_loss = criterion(fake_outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()

        # Update progress bar
        tqdm.write(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], "
                   f"Discriminator Loss: {d_loss.item():.4f}, Generator Loss: {g_loss.item():.4f}")

# Generate and save sample images
z = torch.randn(16, input_size).to(device)
generated_images = generator(z)
torchvision.utils.save_image(generated_images.view(generated_images.size(0), 1, 28, 28),
                             "generated_images.png", nrow=4, normalize=True)

# Generate image based on user input
z_input = torch.randn(1, input_size).to(device)
with torch.no_grad():
    generated_image = generator(z_input)
torchvision.utils.save_image(generated_image.view(1, 1, 28, 28),
                             f"generated_image.png", normalize=True)

Output hidden; open in https://colab.research.google.com to view.